In [44]:
import pandas as pd
import os
import numpy as np
import statsmodels.api as sm

In [45]:
def date2num(date):
    output=[int(''.join(x.split(' ')[0].split('-'))) for x in date]
    return output
find_last=lambda x: x.iloc[-1]

In [46]:
col_name=['date','value']
cost_per_transaction_percent=pd.read_csv('cost_per_transaction_percent.csv',header=None)
cost_per_transaction_percent.columns=col_name
market_price=pd.read_csv('market_price.csv',header=None)
market_price.columns=col_name
n_my_wallet_users=pd.read_csv('n_my_wallet_users.csv',header=None)
n_my_wallet_users.columns=col_name
n_transactions=pd.read_csv('n_transactions.csv',header=None)
n_transactions.columns=col_name


cost_per_transaction_percent['date']=date2num(cost_per_transaction_percent.date)
market_price['date']=date2num(market_price.date)
n_my_wallet_users['date']=date2num(n_my_wallet_users.date)
n_transactions['date']=date2num(n_transactions.date)

cost_per_transaction_percent=cost_per_transaction_percent['value'].groupby(by=cost_per_transaction_percent['date']).apply(find_last).reset_index().set_index('date')
market_price=market_price['value'].groupby(by=market_price['date']).apply(find_last).reset_index().set_index('date')
n_my_wallet_users=n_my_wallet_users['value'].groupby(by=n_my_wallet_users['date']).apply(find_last).reset_index().set_index('date')
n_transactions=n_transactions['value'].groupby(by=n_transactions['date']).apply(find_last).reset_index().set_index('date')

data_all=pd.concat([cost_per_transaction_percent,market_price,n_my_wallet_users,n_transactions],axis=1)
data_all.columns=['cost_per_transaction_percent','market_price','n_my_wallet_users','n_transactions']

data_all['diff_usdprice']=data_all['market_price']-data_all['market_price'].shift(1)
data_all['diff_n_users']=data_all['n_my_wallet_users']-data_all['n_my_wallet_users'].shift(1)
data_all=data_all.dropna()


In [42]:
data_use=data_all.loc[data_all.index<20180501]
x=sm.add_constant(data_use[['diff_usdprice','n_transactions','cost_per_transaction_percent']])
y=data_use['diff_n_users']
lm=sm.OLS(y,x).fit()        
lm.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           diff_n_users   R-squared:                       0.650
Model:                            OLS   Adj. R-squared:                  0.649
Method:                 Least Squares   F-statistic:                     1109.
Date:                Wed, 01 May 2019   Prob (F-statistic):               0.00
Time:                        22:31:12   Log-Likelihood:                -18600.
No. Observations:                1797   AIC:                         3.721e+04
Df Residuals:                    1793   BIC:                         3.723e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
================================================================================================
                                   coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------
const                        -8853.7380    687.241    -12.883      0.000   -1.02e+04   -7505.860
diff_usdprice                    2.9957      0.785      3.815      0.000       1.456       4.536
n_transactions                   0.1210      0.003     47.461      0.000       0.116       0.126
cost_per_transaction_percent  1192.7293    154.794      7.705      0.000     889.133    1496.326
==============================================================================
Omnibus:                     1386.317   Durbin-Watson:                   0.768
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            40532.803
Skew:                           3.357   Prob(JB):                         0.00
Kurtosis:                      25.277   Cond. No.                     7.32e+05
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 7.32e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""